# 上证指数涨跌预测

Install required moduels      

    pip install numpy
    pip install scipy
    pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation

c:\users\hcche\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import peforth
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [12]:
data = pd.read_csv('000777.csv',encoding='gbk',parse_dates=[0],index_col=0)

In [13]:
# data frame 中帶有 None 的 rows 都剔除掉。 本來有 4752 rows 現在只剩 4639 
# 以下很多筆記是根據舊總數寫的，我看除非必要，這一動還是別做，以保持原筆記的易讀。

data = data.replace(to_replace='None', value=np.nan).dropna()  


In [10]:
# pdb.set_trace() # data 本來是新的在上面
data.sort_index(0,ascending=True,inplace=True) # 排序 sort 過後變成最舊的在上面

In [14]:
data.index

DatetimeIndex(['2017-01-20', '2017-01-19', '2017-01-18', '2017-01-17',
               '2017-01-16', '2017-01-13', '2017-01-12', '2017-01-11',
               '2017-01-10', '2017-01-09',
               ...
               '1997-07-23', '1997-07-22', '1997-07-21', '1997-07-18',
               '1997-07-17', '1997-07-16', '1997-07-15', '1997-07-14',
               '1997-07-11', '1997-07-10'],
              dtype='datetime64[ns]', name='日期', length=4639, freq=None)

In [ ]:
data.query('index < "1997-06-20"')

In [ ]:
# raw data has 20 years long, but here only analysis the last 150 days of a chosen day
dayfeature=150
featurenum=5*dayfeature  # 取五個有關係的欄位，特徵值。
x=np.zeros((data.shape[0]-dayfeature,featurenum+1))
y=np.zeros((data.shape[0]-dayfeature));

In [ ]:
peforth.push(data).dictate('constant data');
peforth.push(dayfeature).dictate('constant dayfeature');
peforth.push(x).dictate('value x');
peforth.push(y).dictate('value y');

In [ ]:
%f data :> shape . cr
%f data :> shape[0] . cr
%f data :> shape[0]-v('dayfeature') . cr
peforth.ok();

In [ ]:
# 重疊 shift 150 天的五個「行情數目」加上下一天的開盤 共 751 個「數目」當作 feature X 

for i in range(0,data.shape[0]-dayfeature):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))
    # x[i,featurenum]=data.ix[i+dayfeature][u'开盘价']  # ix deprecated
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']

# 這裡設定要問 AI 的題目

In [ ]:
# 預測當天是否會「漲」

# data.shape[0]-dayfeature 是 4602 同上 x, 所以這是在製作相對所有 x 的 y. x 是 (4602, 751)
# dayfeature 是 150 
# 他用下一天，第 151 天，的「開盤」vs 當天的「收盤」來當作這天是否有漲。這點怪怪的，應該用「第 151 天」
# 的「收盤」來跟「第 150 天」的「收盤」比較才對。

for i in range(0,data.shape[0]-dayfeature):
    if data.iloc[i+dayfeature][u'收盘价']>=data.iloc[i+dayfeature][u'开盘价']:
        y[i]=1
    else:
        y[i]=0          

In [ ]:
# 預測當天是否會「漲停」

# data.shape[0]-dayfeature 是 4602 同上 x, 所以這是在製作相對所有 x 的 y. x 是 (4602, 751)
# dayfeature 是 150 
# 他用下一天，第 151 天，的「開盤」vs 當天的「收盤」來當作這天是否有漲。這點怪怪的，應該用「第 151 天」
# 的「收盤」來跟「第 150 天」的「收盤」比較才對。

for i in range(0,data.shape[0]-dayfeature):
    c1 = data.iloc[i+dayfeature][u'最高价']  # 當天最高價
    c2 = data.iloc[i+dayfeature-1][u'收盘价']  # 前一天的收盤價  
    # or should we use 涨跌幅 directly?
    if (c1-c2)/c2 >= 0.96:
        y[i]=1
    else:
        y[i]=0          

# 這裡選用 classifier 

In [ ]:
# SVM
from sklearn import svm
clf=svm.SVC(kernel='poly')  # so what is SVM? See "11-提交-监督学习 - 课程导学.pdf"
result = []

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
clf=KNeighborsClassifier()
result = []

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
result = []

In [ ]:
# Bayes
from sklearn.naive_bayes import GaussianNB
clf=GaussianNB()
result = []

# 訓練

In [ ]:
for i in range(100):
    x_train, x_test, y_train, y_test = \
                cross_validation.train_test_split(x, y, test_size = 0.2)
    clf.fit(x_train, y_train)
    result.append(np.mean(y_test == clf.predict(x_test)))

In [ ]:
print("svm classifier accuacy:")
print(result)

    --------------- play ground ---------------------

In [ ]:
x_train, x_test, y_train, y_test = \
                cross_validation.train_test_split(x, y, test_size = 0.2)

In [ ]:
%f cr ."  ----------- x_train ------------ " cr
%f __main__ :> x_train . cr
%f __main__ :> x_train type . cr
%f __main__ :> x_train.shape . ."  <-- 這麼多筆的資料，每筆有 150 天，每天 5 個行情數，共 750 個 feature 加 1 個 label " cr
%f __main__ :> x_train[0,0] . space 
%f __main__ :> x_train[0,1] . space
%f __main__ :> x_train[0,749] . space
%f __main__ :> x_train[0,750] . cr
%f cr ."  ----------- y_train ------------ " cr
%f __main__ :> y_train . ."  <-- 看看 y 的樣子 " cr
%f __main__ :> y_train type . cr
%f __main__ :> y_train.shape . cr

%f cr ."  ----------- x ------------ " cr
%f __main__ :> x . cr
%f __main__ :> x type . cr
%f __main__ :> x.shape . ."  <-- 這麼多筆的資料，每筆有 150 天，每天 5 個行情數，共 750 個 feature 加 1 個 label " cr
%f __main__ :> x[0,0] . space 
%f __main__ :> x[0,1] . space
%f __main__ :> x[0,749] . space
%f __main__ :> x[0,750] . cr

%f cr ."  ----------- y ------------ " cr
%f __main__ :> y . ."  <-- 看看 y 的樣子 " cr
%f __main__ :> y type . cr
%f __main__ :> y.shape . cr

In [ ]:
%f 3681 4602 / . cr

    ------------ 研究看漲跌停的趴數是多少？ -----------------

### Sort data['涨跌幅'] 即可知漲跌停趴數

e.g. `data.sort_index(0,ascending=True,inplace=True)` # 排序 sort 過後變成最舊的在上面

以下得知漲跌停幅度是： $\pm 10\%$ 

所謂「涨跌幅」是用「收盤價」來比的：$\frac{現價 - 前一天的收盤價}{前一天的收盤價} \times 100 \%$

弄清楚 000777.csv 上的 涨跌额 涨跌幅 啥定義？對的，正是 （當天收盤─前收盤）/ 前收盤 ＝當天的漲跌幅 所以不用自己算了！

In [ ]:
data.query('index>"1997-07-09" & index<"1997-07-16"')[['收盘价','最高价','最低价','开盘价','涨跌幅']]

In [ ]:
ratio = data['涨跌幅'].sort_values()
%f cr ." 這樣看起來上證的漲停幅度是 10% 實際都不會真的頂到" cr
%f __main__ :> ratio :> [-120:-112] . cr cr 
%f ." 這樣看起來上證的跌停幅度是 -10% 實際都不會真的頂到" cr 
%f __main__ :> ratio :> [2195:2202] . cr 


In [ ]:
data.query('index < "2007-08-15" & index >= "2007-08-10" ')

In [ ]:
data.query('index == "2007-08-13"')['收盘价'] 
data.query('index == "2007-08-10"')['收盘价']

In [ ]:
(34.34-31.22)/31.22 

    ----------- Trace 看他怎麼製作 x feature ---------------
   

In [ ]:
# 結論是： 重疊 shift 150 天的五個「行情數目」加上下一天的開盤 共 751 個「數目」當作 feature X 

%pdb on

for i in range(0,data.shape[0]-dayfeature):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))  # 第 0～149 天的資料，共 150 天 750 個數字
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']  # 第 150 天的開盤，一個數字
    assert(False)
    break

'''
ipdb> type(x)
<class 'numpy.ndarray'>

data.shape is (4752, 14) 每次抓 150 天的資料當作是一筆以便讓 AI 一次看 150 天，讓它觀察其中的端倪
所以 featurenum 是 750
第一筆 data[0:150] 不含第 150 天
data[0:150][[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']] 挑出有關的欄位 shape (150,5)

這兩行是一樣的，用 -1 比較聰明，讓電腦自動決定；用 featurenum 可收檢查的效果，發現潛在問題
np.array(data[i:i+dayfeature][[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum)).shape --> (1,750)
np.array(data[i:i+dayfeature][[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,-1).shape --> (1,750)

x.shape
(4602, 751)   以上 750個數字 加上 1 個數字，共 751 個數字。 給 AI 的一筆輸入資料。

i 最後到 4601 為止，4601 + 150 = 4751 確實是最後一天。
這天的 5 個參數沒有進 x, 只有「開盤價」進了最後一筆 x 的第 751 個 feature.

'''    

In [ ]:

%pdb on
for i in range(0,data.shape[0]-dayfeature):
    if data.iloc[i+dayfeature][u'收盘价']>=data.iloc[i+dayfeature][u'开盘价']:
        y[i]=1
    else:
        y[i]=0          
    assert(False)
    break

'''
data.shape[0]-dayfeature 是 4602 同上 x, 所以這是在製作相對所有 x 的 y. x 是 (4602, 751)
dayfeature 是 150 
他用下一天，第 151 天，的「開盤」vs 當天的「收盤」來當作這天是否有漲。這點怪怪的，應該用「第 151 天」
的「收盤」來跟「第 150 天」的「收盤」比較才對。

i 最後到 4601 為止，4601 + 150 = 4751 確實是最後一天
'''

    -------- 看懂 data.ix[i] 或 data.iloc[i] -----------------------
    其實沒什麼，就是 data[i] 不行，所以要 data.iloc[i] 

In [ ]:
print(3+dayfeature)
%f __main__ :> data :> query('index>"1997-07-09"&index<"1997-07-16"')[['收盘价','最高价','最低价','开盘价','涨跌幅']].iloc[3][u'开盘价'] . cr
%f __main__ :> data :> query('index>"1997-07-09"&index<"1997-07-16"')[['收盘价','最高价','最低价','开盘价','涨跌幅']].ix[3][u'开盘价'] . cr        

%f cr cr ." 以下這三個都一樣意思，就是取 data-frame 的哪幾 row " cr
data.ix[:2]
data.iloc[:2]     
data[:2]        # 這樣用與 ix() iloc() 無異

%f cr cr ." 但是 data[1] 就不行了！！必須透過 data.iloc[1] " cr
data.iloc[1]    
type(data.iloc) # pandas.core.indexing._iLocIndexer 特殊型態

In [8]:
# df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data = data.replace(to_replace='None', value=np.nan).dropna()


,股票代码,名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,换手率,成交量,成交金额,总市值,流通市值
日期,,,,,,,,,,,,,,
1997-07-10,'000777,中核苏阀,9.06,10.33,8.80,10.10,5.33,3.73,69.9812,62.3698,16839842,1.553789e+08,6.795000e+08,2.446200e+08
1997-07-11,'000777,中核苏阀,9.97,9.97,8.48,8.75,9.06,0.91,10.0442,42.3227,11427140,1.085246e+08,7.477500e+08,2.691900e+08
1997-07-14,'000777,中核苏阀,10.97,10.97,10.33,10.35,9.97,1.0,10.0301,46.358,12516652,1.350109e+08,8.227500e+08,2.961900e+08
1997-07-15,'000777,中核苏阀,11.53,11.80,10.81,11.30,10.97,0.56,5.1048,31.2442,8435941,9.504755e+07,8.647500e+08,3.113100e+08
1997-07-16,'000777,中核苏阀,10.98,11.50,10.90,11.49,11.53,-0.55,-4.7702,16.7164,4513434,5.010548e+07,8.235000e+08,2.964600e+08
1997-07-17,'000777,中核苏阀,10.99,11.26,10.52,10.81,10.98,0.01,0.0911,12.9689,3501611,3.830279e+07,8.242500e+08,2.967300e+08
1997-07-18,'000777,中核苏阀,11.10,11.12,10.50,10.80,10.99,0.11,1.0009,9.4632,2555060,2.768096e+07,8.325000e+08,2.997000e+08
1997-07-21,'000777,中核苏阀,11.20,11.78,11.08,11.28,11.10,0.1,0.9009,14.888,4019750,4.607238e+07,8.400000e+08,3.024000e+08
1997-07-22,'000777,中核苏阀,11.96,12.10,11.23,11.23,11.20,0.76,6.7857,23.5468,6357648,7.452792e+07,8.970000e+08,3.229200e+08


In [34]:
np.max(data['涨跌幅'])
# np.min(float(data['涨跌幅']))
float(data['涨跌幅'].iloc[3])
max([float(i) for i in data['涨跌幅']])  # 這個會跑出 69.9812 所以前一天沒有交易的也要剔除
min([float(i) for i in data['涨跌幅']])  # 這個會跑出 -37.5388 也很奇怪

'9.9975'

1.0402

69.9812

-37.5388

In [20]:
min(-9.3, -0.1)

-9.3